Group:

1) Last Name, First Name, Student Number:

2) Last Name, First Name, Student Number:


Today's goal is to familiar yourself with the concept of segmentation and handling of colors images. To do so, the objective of this laboratory is to transform a real picture into a hand-drawn equivalent. Two main steps are invoved in the operation: In the first part, we will focus on edge detection (as a proxy for the lines of our drawing). The second part will be focused on smartly reducing the number of colors in the image.


**Due date**: 

Assignment due date is for Monday December 7th, at 11h55 PM. A 3 points penalty per day will be applied in case of delay.

**Submission files** :

All code must be contained in the present template, as well as the answers to the questions (either commented within the code or with special Markdown/text cells).
Please follow the order of the subject.
Commenting the code is important and the overall clarity of the work will be taken in account. Make sure that every variable is clearly understandable and every figure readable.

You will also have to submit a static **HTML** version of this notebook *File->Download as...->HTML*. Put all your files (ipynb, html, eventually others externals ones) in a single **.zip** archive, named after your student number (StudentNb1_StudentNb2.zip).


# Imports et utilitaires

In [1]:
import cv2
import numpy as np
import scipy
import matplotlib.pyplot as plt
plt.rcParams['image.cmap'] = 'gray' # Default color map, do not change it
import matplotlib
import scipy.signal
matplotlib.rcParams['figure.figsize'] = (25.0, 10.0) # Default figure siz, change it only if relevant.
from scipy.cluster.vq import kmeans

def imshow(img, title=None, ax=None, is_bgr=False, cmap='gray'):
    """
    param img: image to display (either hxwx3 or hxw)
    param title: Figure's title
    param ax: Axe on which the image will be shown. If none is given, a new axe will be created.
    param is_bgr: If true, will convert the image from BGR to RGB
    param cmap: Color map.
    """
    show=False
    plt.axis('off')
    if ax is None:
        show=True
        if title is not None:
            plt.title(title)
        ax = plt
    else:
        if title is not None:
            ax.set_title(title)
            ax.set_axis_off()
    
    if img.ndim==2:
        ax.imshow(img, cmap='gray')
    else:
        if is_bgr:
            img = img[:,:,::-1].copy()
        ax.imshow(img)
    if show:
        plt.show()

def f2int(img):
    return (img*255).astype(np.uint8)

def int2f(img):
    return img.astype(np.float32)/255.

# Part 1
Drawing implies lines. The first part of the lab is dedicated to edge detection in the image. To display the image, you are free to use the function **imshow** given in the previous cell.

## Question 1



Load the the **chaton.png** ("chaton"=kitten), convert it into float and display it.



Make sure that the colors are correctly rendered (as in your viewer) and that the pixels' values are inside the range [0,1].

> **Warning** If you use **openCV** (cv2), the default channels' order is <span style="color:blue">b</span>, <span style="color:green">g</span>, <span style="color:red">r</span> instead of the conventional <span style="color:red">r</span>, <span style="color:green">g</span>, <span style="color:blue">b</span>.

> In most of this lab, we will work  with image of type **np.float32**. But some functions expect images in the range [0, 255] with uint8 dtype. To make it easier for you, we provide two functions **f2int** and **int2f** that'll let you switch between both types (see above cell).


## Question 2
The lightness of an image is defined by:
$$
c = \frac{1}{2}(\max(r, g, b) + \min(r, g, b))
$$

Compute and display the lightness of the image.

## Question 3
We will now focus on edge detection. Most of the methods you've seen during the course are very sensitive to noise. In our case, the kitten's fur can be assimilated to a pepper and salt noise. What type of filter should you use to attenuate it? Filter the lightness map witht this filter using a window size of 7. 


**Warning** For the next questions, we will compare different edge detections methods. For each method, you'll have to use the filtered lightness map from the previous question.

# Question 4

Compute and display on two adjacents graphs (subplots):
- The image of the horizontal gradient
- The image of the vertical gradient

On a third graph (new figure), display the magnitude of the gradient

> To compute the gradient, you'll have to define yourself the convolution mask. For the convolution itself,you can use the function **scipy.signal.convolve2d**:
```python
convolved = scipy.signal.convolve2d(img, mask, mode='same')
```

# Question 5

Redo the same thing, but this time by pre-filtering the input lightness with a Gaussien  filter with a standard deviation $\sigma=1$. You'll have to code yourself the gaussian mask (You can not use cv2.GaussianBlur). Justify the size of the mask (relatively to $\sigma$).
What happens when we vary $\sigma$?

> As a reminder, a  2D Gaussian is defined by 
$$G(x, y)=A e^{\frac{-(x²+y²)}{2\sigma²}}$$
where $A$ is a constant such that the area under the Gaussian is equal to 1.

# Question 6
In order to extract the edges from the magnitude image, we will use Otsu's method. You'll have to implement it.
Complete the following function:
```python
def otsu_thresholding(img):
    assert img.dtype==np.uint8, "The image image must have a uint8 type"
    thresholds = np.unique(img)
    hist, bins = np.histogram(img, np.arange(256))
    bins = bins[:-1]
    hist = hist/img.size
    nu = 0
    for k in thresholds:
        hist_c1  = hist[:k] # Histogram of cluster 1 
        hist_c2  = hist[k:] # Histogram of cluster 2
        P1_k = ... # Probability to belong to cluster 1
        P2_k = ... # To complete
        if P1_k==0:
            continue
        if P2_k==0:
            break
        m1_k =  np.sum(bins[:k]*hist_c1) / P1_k.astype(np.float32) # Mean on cluster 1
        m2_k =  ... # To complete
        var_interclasse = ... # To complete
        if var_interclasse>nu:
            threshold = k
            nu = var_interclasse
    thresholded = ... # To complete
    return thresholded
```
that takes a monochromatic image as a parameter and automatically threshold it using Otsu's method. 

> Careful, for Otsu's method, the image must be of type uint8.

Compare Otsu's thresholding on the magnitude with and without the Gaussian prefiltering. What do you observe?

## Question 7
Let's now try edge detection using Marr-Hildreth's approach. A quick reminder of the method:
* The image is filtered with a gaussian (parametrized by $\sigma$)
* The Laplacian is computed
* We search the locations of the zero croissing of the Laplacian. We only keep the ones with an absolute difference greather than a given threshold $t<1$


> The code  for this last step is given in the function below. It expects the Laplacian as an input as well as a treshold.


Display the edges for $\sigma=5$ and a threshold equal to 0.005. For the computation of the Laplacian, you need to create yourself the corresponding filter mask.
Test the edge detection with different values of $\sigma$ and threhsold seuil and indicate the effect of both parameters.

In [ ]:
def zeros_crossing(img, threshold):
    out = np.zeros_like(img)
    for i in range(1, img.shape[0]-1):
        for j in range(1, img.shape[1]-1):
            ec = 0
            if img[i-1, j]*img[i+1, j] < 0: # y
                ec = max(ec, np.abs(img[i+1, j]-img[i-1, j]))
            if img[i, j-1]*img[i, j+1] < 0: # x
                ec = max(ec, np.abs(img[i, j+1]-img[i, j-1]))
            if img[i-1, j-1]*img[i+1, j+1] < 0: # diag
                ec = max(ec, np.abs(img[i+1, j+1]-img[i-1, j-1]))
            if img[i+1, j-1]*img[i-1, j+1] < 0: # other diag
                ec = max(ec, np.abs(img[i+1, j-1]-img[i-1, j+1]))
            
            out[i, j] = ec
    
    threshold = threshold*np.max(out)
    return out>threshold


## Question 8

Finally, you'll try Canny's approach to edge detection. You are not asked to implement it yourself: you can use the function from **openCV** in the following way:

```python
edges_canny = cv2.Canny(clarte, min_hysteresis, max_hysteresis) # lightness must be in uint8
```
where min_hysteresis, max_hysteresis represent respectively the lower and upper hysteris thresholds.
Using a upper threshold to 100, compared the effects of differents values for the lower threshold. Discuss the results.

## Question 9
Display on a 2x2 subplots the edges obtained with the 4 differents approaches. Comments the results of each method and their respective flexibility.


# Part 2

This part work on colors images. Therefore, you should start from the very first image loaded in question 1.

## Question 10
Load the image **chaton.png**, filter with a median-filter of size 9, convert it into float and display it. 

A picture contains much more different colors than a drawing. We will therefore work on an operation called  **posterization** that consist in reducing the number of colors contained in an image. We can do so in differents ways.

### Naive posterization

In this approach, we recense all the N unique colors in the image as an array $T_1$ with shape $N\times 3$. Then we keep only the first K most represented colors such that we obtain a new array $T_2$ with shape $K \times 3$. 

For each pixel of the image, we replace the original color as its closest one in $T_2$. Proximity is defined by using the euclidean distance.



## Question 11
Implement the function
```python 
def recense_colors(img):
    ...
    return unique_colors, count 

```
That returns an array $T1$ of unique colors as well as the occurence of each one in the image.

## Question 12
Complete the following function
```python
def euclidean_distance(col1, col2):
    """
    This function takes two colors arrays and return the table of all the distances between them
    :param col1: Array of shape Nx3
    :param col2: Array of shape Mx3
    :return d: Array of euclidean distances of shape NxM

    """
    ...
    return d

def naive_posterization(img, K=32):
    h, w, c = img.shape
    unique_colors, counts = recense_colors(img)
    T2 = ... # To complete: we only keep the K most frequent colors
    distances = euclidean_distance(T2,  img.reshape(-1, c))
    indices_d_min = ... # For each pixels, we recover the value of the indice that will give us the new color in T2.
    posterization = ...
    return posterization.reshape(h, w, c)
    
```

## Question 13
Test and display the result of the posterization with K=64. What do you deduce from the limitations of the naive posterization?

## Question 14
To correct the defaults from the naive posterization, we will use a more sophisticated approach to reduce the number of colors. Instead of only keeping the most frequent ones, we will regroup the colors within K clusters in a way that each cluster contains colors that are close to each others. Our new array $T2$ will be composed of the mean of each cluster. This fundamental algorithm in AI is called K-mean. 

No worries, you don't have to implement it! You can simply use the following function:
```python
from scipy.cluster.vq import kmeans
T2 = kmeans(T1, K)[0] # K is the number of colors you want to keep and T1 the array of unique colors..
```

Implement the function
```python
def posterizations_kmeans(img, K=8):
    ...
```
that uses kmeans to compute T2 and posterizes the image accordingly. Display the results for K=2,K=8 et K=64. Conclusions?

## Question 15

In the previous two cases, we have posterized the image globally without considering the fact that some colors are more important than others. Nonetheless, on a drawing, there are much more different hues than saturations or values (if you take a pencil box, you usually have about ten different hues but barely  two or three saturation/values per hue). 

Let's mimic this by posterizing our channels independantly. Taking inspiration from previous questions, implement the function:
```python
def posterize_grayscale(canal, K=8):
    ...
```
that posterizes a single channel using k_means approach.



## Question 16
For now, we will use a color space very similar to the HSI (seen in class) which is the HSV (Hue/Saturation/Value). 
To convert a RGB into HSV et reciprocally, use the following functions:
```python
hsv = cv2.cvtColor(rgb, cv2.COLOR_RGB2HSV)
rgb = cv2.cvtColor(hsv, cv2.COLOR_HSV2RGB)

```
Display separately the three channels H, S and V and comment the results. In which range are each channel?

## Question 17
Posterize each channel independantly such that:
- Hue contains 8 differents elements
- Saturation only 3
- Value only 3

Then reconstruct the image in the RGB color space (display it).

What is the thoerical maximum RGB colors that the image can contain under those constraints? And in practise, how many colors are there after this posterization?

## Question 18
Let's finish our drawing by adding edges! We will use the ones obtained in the first part.

> To do so, you have many options: indexation (set to 0 the colors of the images in the edges), piecewise multiplication, bitwise_xor... You're free to chose the approach you like the most!


Create four subplots and display the differents edges (gradients, canny, LoG...) in back noir on top of the posterized image from previous question.
Which one seems the most convincing for our application?

## Question 19
In reality, most lines of a drawing are rarely perfectly black; their colors may depend of their widths and the colors of the object they delimit.
By taking in account these two considerations, propose a way to improve our drawing effect. You can combine as you want any of our previously obtained results.

# Question bonus

We provide an extension of the last image, on which we add a final effect. Load the image **dessin_chaton.png** and display it.
Can you identify the effect added? Propose (and test) a simple way to reproduce it. 